In [2]:
from neo4j import GraphDatabase
import pandas as pd
import json
from tqdm import tqdm

In [4]:
port = 7687 # Check if is the case for your server!
PWD='knowledge'


In [5]:
# Database wrapper from: https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4


class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


conn = Neo4jConnection(uri="bolt://localhost:"+str(port),
                       user="neo4j",
                       pwd=PWD)

In [6]:
import time


def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.

    total = 0
    batch = 0
    start = time.time()
    result = None

    for batch in tqdm(range(len(rows) // batch_size + 1)):
        batch_start = time.time()
        res = conn.query(query,
                         parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        result = {"batch_size": batch_size,
                  "batches_done": batch,
                  "batch_time": time.time() - batch_start,
                  "total_time": time.time()-start}
        # print(result)

    return result

def add_papers(rows, batch_size=5000):
   # Adds paper nodes and relationships (:Author)-[:AUTHORED]-(:Paper), (:Paper)-[:REFERENCES]-(:Paper)
   query = '''
    // Create papers
    UNWIND $rows as paper
    MERGE (p:Paper {paperid: paper.id})
    ON CREATE SET
    p.title = paper.title,
    p.year = paper.year,
    p.n_citation = paper.n_citation,
    p.doi = paper.doi

    // Match authors
    WITH paper, p
    UNWIND  paper.authors AS author
    MERGE (a:Author {authorid: author.id})
    ON CREATE SET a.name = author.name
    MERGE (a)-[:AUTHORED]->(p)

    // Match references
    WITH paper, p
    UNWIND  paper.references AS refid
    MATCH (r:Paper {paperid:refid})
    MERGE (p)-[:REFERENCES]->(r)
    RETURN count(p:Paper) as total
   '''

   return insert_data(query, rows, batch_size)

In [7]:
conn.query('CREATE INDEX paper_id_index IF NOT EXISTS FOR (p:Paper) ON (p.paperid);')
conn.query('CREATE INDEX author_id_index IF NOT EXISTS FOR (a:Author) ON (a.authorid);')

[]

# Add paper nodes

In [ ]:
file = "data/dblp_papers_v11.txt"
n_papers = 4107340 # Number of papers in the dataset
subset = ["id", "title", "year", "n_citation", "doi", "authors", "references"]

# TODO: Add tqdm max of the number of papers
with open(file, 'r') as f:
    for episode in tqdm(range(n_papers // 100000)):
        try:
            lines = 100000
            rows  = []
            for line in f:
                rows.append(json.loads(line))
                lines -= 1
                if lines == 0: break
            df = pd.DataFrame(rows)
            add_papers(df[subset], 1000)
        except Exception as e:
            print(e)
            break

 41%|████▏     | 17/41 [12:31<20:13, 50.54s/it]

# Load stackexchange XML

In [25]:
import re


def process_stackexchange(board: str, verbose=0) -> list[pd.DataFrame]:
    """_summary_

    Args:
        board (str): Stackexchange board to load. Expected file structure is
            ./data/`board`/*.xml
        verbose: Prints debug steps if set to a value higher than 0.

    Returns:
        list[pd.DataFrame]: List of pandas dataframes with Comments and
            PostHistory data with extracted DOIs and markdown links.
    """
    # Regex patterns
    DOI_PATTERN = "10\.\d{4,9}/[-._;\(\)/:A-Z0-9]+[/A-Z0-9]"
    MD_PATTERN = "\[([\w\s\d]+)\](https?:\/\/[\w\d./?=#]+)"

    def process_set(set: str) -> pd.DataFrame:
        """Processes single file in board data

        Args:
            set (str): One of "Comments" or "PostHistory"

        Returns:
            pd.DataFrame: Original data with extracted DOIs and Markdown links
        """
        # Load xml file
        path = f"data/{board}/{set}.xml"
        with open(path, encoding="utf8") as file:
            df = pd.read_xml(file.read())

        # Extract all DOIs using pattern defined before to list per row
        if "Text" in df.columns:
            df["DOIs"] = df["Text"].apply(lambda text: re.findall(DOI_PATTERN, str(text)))
            # df = df.loc[df["DOIs"].str.len() > 2]
        df["board"] = board
        if verbose > 0:
            print(f" --- {board}/{set} loaded:")
            print(df.head())
            print(df.shape)
        return df

    return process_set("Comments"), process_set("Posts"), process_set("PostLinks")

In [9]:
def add_boards(rows, batch_size=50):
    # Adds board nodes
    query = '''
    UNWIND $rows as board
    MERGE (b:Board {boardname: board.name})
    RETURN count(b:Board) as total
    '''
    return insert_data(query, rows, batch_size)

In [23]:
def add_posts(rows, batch_size=50):
   # Adds post nodes
   query = '''
    // Create posts
    UNWIND $rows as post
    MERGE (p:Post {postid: post.Id})
    ON CREATE SET
    p.title = post.Title,
    p.text = post.Body,
    p.tags = post.Tags,
    p.score = post.Score

    WITH post, p
    UNWIND post.DOIs AS doi
    MATCH (r:Paper {doi:doi})
    MERGE (p)-[:REFERENCES]->(r)

    WITH post, p
    MATCH (b:Board {boardname:post.board})
    MERGE (p)-[:FROM_BOARD]->(b)
    RETURN count(p:Post) as total
   '''
   return insert_data(query, rows, batch_size)

In [11]:
def add_comments(rows, batch_size=50):
   # Adds comment nodes
   query = '''
    // Create comments
    UNWIND $rows as comment
    MERGE (c:Comment {commentid: comment.Id})
    ON CREATE SET
    c.text = comment.Text

    // Match posts
    WITH comment, c
    MATCH (p:Post {postid: comment.PostId})
    MERGE (c)-[:RESPONDS_TO]->(p)

    // Match references
    WITH comment, c
    UNWIND comment.DOIs AS doi
    MATCH (r:Paper {doi:doi})
    MERGE (c)-[:REFERENCES]->(r)
    RETURN count(c:Comment) as total
   '''
   return insert_data(query, rows, batch_size)

In [12]:
def add_post_links(rows, batch_size=50):
   # Adds post nodes
   query = '''
    // Create posts
    UNWIND $rows as post_link
    MATCH (p:Post {postid:post_link.PostId})
    MATCH (r:Post {postid:post_link.RelatedPostId})
    MERGE (p)-[:REFERENCES]->(r)
    RETURN count(p:Post) as total
   '''
   return insert_data(query, rows, batch_size)

In [13]:
conn.query('CREATE INDEX paper_doi_index IF NOT EXISTS FOR (p:Paper) ON (p.doi);')
conn.query('CREATE INDEX post_id_index IF NOT EXISTS FOR (p:Post) ON (p.postid);')
conn.query('CREATE INDEX comment_id_index IF NOT EXISTS FOR (c:Comment) ON (c.commentid);')
conn.query('CREATE INDEX board_index IF NOT EXISTS FOR (b:Board) ON (b.boardname);')

[]

In [29]:
se_boards = [
    # "ai",
    # "cstheory",
    "datascience",
    "stats",
]
add_boards(pd.DataFrame({"name": se_boards}), batch_size=1)

100%|██████████| 3/3 [00:00<00:00, 742.88it/s]


{'batch_size': 1,
 'batches_done': 2,
 'batch_time': 0.0005042552947998047,
 'total_time': 0.005553007125854492}

In [30]:
for board in se_boards:
    print("Board:", board)
    comments_df, posts_df, postlinks_df = process_stackexchange(board)
    add_posts(posts_df, batch_size=100)
    add_comments(comments_df, batch_size=100)
    add_post_links(postlinks_df, batch_size=100)
    del(comments_df, posts_df)

Board: datascience


100%|██████████| 33/33 [00:00<00:00, 119.20it/s]


Board: stats


100%|██████████| 898/898 [00:04<00:00, 184.15it/s]


In [ ]:
def query_for_use_case_1(conn: Neo4jConnection, post_title: str):
   query = '''
    MATCH (post1:Post)
    WHERE post1.title CONTAINS ''' + post_title + '''
    MATCH (post1)-[:REFERENCES]->(paper:Paper)

    WITH paper
    MATCH (post2: Post)-[:REFERENCES]->(paper)
    RETURN post2
   '''
   return conn.query(query)

In [68]:
conn.query('CREATE INDEX paper_doi_index IF NOT EXISTS FOR (p:Paper) ON (p.doi);')
conn.query('CREATE INDEX post_id_index IF NOT EXISTS FOR (p:Post) ON (p.postid);')
conn.query('CREATE INDEX comment_id_index IF NOT EXISTS FOR (c:Comment) ON (c.commentid);')
conn.query('CREATE INDEX board_index IF NOT EXISTS FOR (b:Board) ON (b.boardname);')

[]

In [67]:
se_boards = [
    # "ai",
    # "cstheory",
    #  "datascience",
     "stats",
     ]
add_boards(pd.DataFrame({"name": se_boards}), batch_size=1)

100%|██████████| 2/2 [00:00<00:00, 168.03it/s]


{'batch_size': 1,
 'batches_done': 1,
 'batch_time': 0.002944469451904297,
 'total_time': 0.013958215713500977}

In [70]:
for board in se_boards:
    print("Board:", board)
    comments_df, posthistory_df, postlinks_df = process_stackexchange(board)
    posthistory_df = posthistory_df.sort_values('CreationDate', ascending=False).drop_duplicates(subset=["PostId"])
    add_posts(posthistory_df, batch_size=100)
    add_comments(comments_df, batch_size=100)
    add_post_links(postlinks_df, batch_size=100)
    del(comments_df, posthistory_df)

Board: stats
 --- stats/Comments loaded:
   Id  PostId  Score                                               Text  \
0   1       3      6  Could be a poster child fo argumentative and s...   
1   2       5      0          Yes, R is nice- but WHY is it 'valuable'.   
2   3       9      0  Again- why?  How would I convince my boss to u...   
3   4       5     11  It's mature, well supported, and a standard wi...   
4   6      14      9  why ask the question here?  All are community-...   

              CreationDate  UserId ContentLicense UserDisplayName DOIs  board  
0  2010-07-19T19:15:52.517    13.0   CC BY-SA 2.5            None   []  stats  
1  2010-07-19T19:16:14.980    13.0   CC BY-SA 2.5            None   []  stats  
2  2010-07-19T19:18:54.617    13.0   CC BY-SA 2.5            None   []  stats  
3  2010-07-19T19:19:56.657    37.0   CC BY-SA 2.5            None   []  stats  
4  2010-07-19T19:22:27.947    23.0   CC BY-SA 2.5            None   []  stats  
(725176, 10)


MemoryError: 